In [ ]:
import pandas as pd
from prophet import Prophet

import seaborn as sns
import matplotlib.pyplot as plt

from prophet.plot import plot_plotly, plot_components_plotly

import plotly.io as pio
pio.renderers.default = "notebook"

plt.style.use('Solarize_Light2')

In [ ]:
df = pd.read_csv('../data/processed_sensor_dwd_train.csv', index_col=0)

In [ ]:
# add sensor IDs
df_location = df.groupby(['lat', 'lon']).count().reset_index()[['lat', 'lon']]
df_location['location_id'] = df_location.index+1
df = df.merge(df_location, on=['lat', 'lon'], how='left')

In [ ]:
df

In [ ]:
df_prophet = df.query('city == "Bremen"')
df_prophet = df_prophet[['timestamp','PM2p5','location_id']]
df_prophet.rename(columns={'timestamp': 'ds', 'PM2p5': 'y'}, inplace=True)

df_prophet.dropna(inplace=True)

location_list_bremen = df_prophet['location_id'].unique().tolist()

df_prophet.location_id.nunique()
df_prophet.isna().sum()
df_prophet

In [ ]:
prophet_models_per_location = {}

for n, location_id in enumerate(location_list_bremen):
    print('-----'*10)
    print(f'no: {n + 1}, location_id: {location_id}')
    df_location = df_prophet[df_prophet['location_id'] == location_id]
    df_location.drop('location_id', axis=1, inplace=True)
    model = Prophet()
    model.fit(df_location)

    prophet_models_per_location[location_id] = model

In [ ]:
prophet_forecasts_per_location = {}
for location_id in location_list_bremen:
    print('-----'*10)
    print(f'location_id: {location_id}')
    model = prophet_models_per_location[location_id]
    
    future = model.make_future_dataframe(periods=240, freq='H')
    forecast = model.predict(future)

    prophet_forecasts_per_location[location_id] = forecast

In [ ]:
location_list_bremen

In [ ]:
for location_id in location_list_bremen:
    model = prophet_models_per_location[location_id]
    fig1 = model.plot(prophet_forecasts_per_location[location_id])


In [ ]:
for location_id in location_list_bremen:
    model = prophet_models_per_location[location_id]
    fig2 = model.plot_components(prophet_forecasts_per_location[location_id])

In [ ]:
#for location_id in location_list_bremen:
model = prophet_models_per_location[125]
plot_plotly(model, forecast) 